In [1]:
/* 24-13 */
-- Create database and turn auto create statistics off
USE master;
GO

Commands completed successfully.

Total execution time: 00:00:00.001

In [2]:
ALTER DATABASE AdventureWorks2014 
SET AUTO_CREATE_STATISTICS OFF WITH NO_WAIT
GO

Commands completed successfully.

Total execution time: 00:00:00.002

In [3]:
USE master;
GO
-- Create the Event Session
IF EXISTS(SELECT * 
          FROM sys.server_event_sessions 
          WHERE name='MissingColumnStats')
    DROP EVENT SESSION MissingColumnStats 
    ON SERVER;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.005

In [4]:
-- Create XEvent session
CREATE EVENT SESSION [MissingColumnStats] ON SERVER 
ADD EVENT sqlserver.missing_column_statistics(SET collect_column_list=(1)
ACTION(sqlserver.sql_text, sqlserver.database_name)) 
ADD TARGET package0.event_file(SET filename=N'C:\Database\XE\MissingColumnStats.xel')
GO

Commands completed successfully.

Total execution time: 00:00:00.009

In [5]:
--Start XEvent session
ALTER EVENT SESSION [MissingColumnStats]
ON SERVER
STATE = START
GO

: Msg 25602, Level 17, State 0, Line 2
The target, "5B2DA06D-898A-43C8-9309-39BBBE93EBBD.package0.event_file", encountered a configuration error during initialization. Object cannot be added to the event session. The operating system returned error 3: 'The system cannot find the path specified.
' while creating the file 'C:\Database\XE\MissingColumnStats_0_132477023748090000.xel'.

Total execution time: 00:00:00.008

In [6]:
/* baseline our stats */
USE AdventureWorks2014;
GO
SELECT *
	FROM sys.stats
	WHERE OBJECT_NAME(object_id) = 'SalesOrderDetail';
GO

Commands completed successfully.

(13 rows affected)

Total execution time: 00:00:00.033

object_id,name,stats_id,auto_created,user_created,no_recompute,has_filter,filter_definition,is_temporary,is_incremental,has_persisted_sample,stats_generation_method,stats_generation_method_desc
1154103152,PK_SalesOrderDetail_SalesOrderID_SalesOrderDetailID,1,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,AK_SalesOrderDetail_rowguid,2,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,IX_SalesOrderDetail_ProductID,3,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000006_44CA3770,4,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000007_44CA3770,5,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,NCI_UnitPrice_SalesOrderDetail,6,0,0,0,1,([UnitPrice]>=(150.00) AND [UnitPrice]<=(175.00)),0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000004_44CA3770,7,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,NCI_ProductID_SalesOrderDetail,8,0,0,0,1,"([ProductID] IN ((776), (777)))",0,0,0,0,Sort based statistics
1154103152,NCI_SalesOrderDetail_CarrierTrackingNumber,9,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,SalesOrderDetailID,11,0,1,0,0,NULL,0,0,0,0,Sort based statistics


In [7]:
/* Run a query that should generate some statistics need */
USE AdventureWorks2014;
GO
Select Unitprice
	From Sales.SalesOrderDetail 
	WHERE UnitPrice >= 1000.00 AND UnitPrice <= 1500.00;
GO

Commands completed successfully.

(9582 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.119

Unitprice
1229.4589
1229.4589
1242.8518
1242.8518
1242.8518
1466.0100
1308.9375
1466.0100
1308.9375
1466.0100


In [8]:
/* Confirm no new stats */
SELECT *
	FROM sys.stats
	WHERE OBJECT_NAME(object_id) = 'SalesOrderDetail';
GO

(13 rows affected)

Total execution time: 00:00:00.043

object_id,name,stats_id,auto_created,user_created,no_recompute,has_filter,filter_definition,is_temporary,is_incremental,has_persisted_sample,stats_generation_method,stats_generation_method_desc
1154103152,PK_SalesOrderDetail_SalesOrderID_SalesOrderDetailID,1,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,AK_SalesOrderDetail_rowguid,2,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,IX_SalesOrderDetail_ProductID,3,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000006_44CA3770,4,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000007_44CA3770,5,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,NCI_UnitPrice_SalesOrderDetail,6,0,0,0,1,([UnitPrice]>=(150.00) AND [UnitPrice]<=(175.00)),0,0,0,0,Sort based statistics
1154103152,_WA_Sys_00000004_44CA3770,7,1,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,NCI_ProductID_SalesOrderDetail,8,0,0,0,1,"([ProductID] IN ((776), (777)))",0,0,0,0,Sort based statistics
1154103152,NCI_SalesOrderDetail_CarrierTrackingNumber,9,0,0,0,0,NULL,0,0,0,0,Sort based statistics
1154103152,SalesOrderDetailID,11,0,1,0,0,NULL,0,0,0,0,Sort based statistics


In [9]:
/* parse the event data */
use master;
GO

SELECT
event_data.value('(event/@name)[1]', 'varchar(50)') AS event_name,
    event_data.value('(event/@timestamp)[1]', 'varchar(50)') AS [TIMESTAMP],
	event_data.value('(event/action[@name="database_name"]/value)[1]', 'varchar(max)') AS DBName
	,event_data.value('(event/action[@name="sql_text"]/value)[1]', 'varchar(max)') AS SQLText
	,event_data.value('(event/data[@name="column_list"]/value)[1]', 'varchar(max)') AS AffectedColumn
FROM(
SELECT CONVERT(XML, t2.event_data) AS event_data
 FROM (
  SELECT target_data = convert(XML, target_data)
   FROM sys.dm_xe_session_targets t
    INNER JOIN sys.dm_xe_sessions s 
        ON t.event_session_address = s.address
   WHERE t.target_name = 'event_file'
    AND s.name = 'MissingColumnStats') cte1
   CROSS APPLY cte1.target_data.nodes('//EventFileTarget/File') FileEvent(FileTarget)
   CROSS APPLY  sys.fn_xe_file_target_read_file(FileEvent.FileTarget.value('@name', 'varchar(1000)'), NULL, NULL, NULL) t2)
    AS evts(event_data);

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.153

event_name,TIMESTAMP,DBName,SQLText,AffectedColumn
